In [39]:
import os, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# I. Scrape and Save reddit data to folders
        edit your credentials in ScrapeRedditData.py before running

In [1]:
from ScrapeRedditData import scrapeSubreddit

university_subreddits = [ "CMU", "drexel", "temple", "upenn", "villanova"]
nonuniversity_subreddits = [ "investing", "AskNYC", "thingsmykidsaid", "nfl", "help"]

for university_subreddit in university_subreddits:
    scrapeSubreddit(subreddit_name=university_subreddit, file_directory='./data/university_data/')

for nonuniversity_subreddit in nonuniversity_subreddits:
    scrapeSubreddit(subreddit_name=nonuniversity_subreddit, file_directory='./data/nonuniversity_data/')

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


CMU:100
drexel:100
temple:73
upenn:100
villanova:100


# II. Collect post meta data and parse through post 'title', 'user', and 'text'
        text files are line separated, with first two lines being the title and user name - "title\n user\n text"

In [21]:
nonUniFiles=os.listdir('./data/nonuniversity_data/')
uniFiles=os.listdir('./data/university_data/')

df_dict = {'titles':[], 'user':[], 'text':[], 'uni_label':[], 'reddit_label':[]}

# non-University
for nonUFile in nonUniFiles:
    
    # Handle dirty file contents
    if nonUFile == '.DS_Store' or nonUFile == '.ipynb_checkpoints':
        pass
    
    else:
        # Subreddit name is the prefix of file name
        df_dict['reddit_label'].append(re.findall('(\w+)_',nonUFile)[0])
        df_dict['uni_label'].append('non_uni')

        with open('./data/nonuniversity_data/'+nonUFile, 'r') as f:
            p=f.readlines()
            df_dict['titles'].append(p[0].strip())
            df_dict['user'].append(p[1].strip())
            df_dict['text'].append(''.join(p[2:]).replace('\n', ' '))

# University
for uFile in uniFiles:
    
    # Handle dirty file contents
    if uFile == '.DS_Store' or nonUFile == '.ipynb_checkpoints':
        pass
    
    else:
        # Subreddit name is the prefix of file name
        df_dict['reddit_label'].append(re.findall('(\w+)_',uFile)[0])
        df_dict['uni_label'].append('uni')

        with open('./data/university_data/'+uFile, 'r') as f:
            p=f.readlines()
            df_dict['titles'].append(p[0].strip())
            df_dict['user'].append(p[1].strip())
            df_dict['text'].append(''.join(p[2:]).replace('\n', ' '))
            
            
df_LARGE = pd.DataFrame(df_dict)

# III. Convert text and titles to TF-IDF vectors and save to csv for modeling
        Also save the raw text and label data in separate CSVs

In [31]:
tf_idf_vectorizer = TfidfVectorizer(max_features=1000)

tf_idf_text = tf_idf_vectorizer.fit_transform(df_LARGE['text']).toarray()
tf_idf_titles = tf_idf_vectorizer.fit_transform(df_LARGE['titles']).toarray()

np.savetxt('./data/tf_idf_text.csv', X=tf_idf_text, delimiter=',')
np.savetxt('./data/tf_idf_titles.csv', X=tf_idf_titles, delimiter=',')



df_LARGE[['uni_label', 'reddit_label']].to_csv('./data/y_data.csv', index=False)
df_LARGE[['text']].to_csv('./data/raw_text.csv', index=False)